# Box pendulum
- This is modelling for the rotary pendulum

In [1]:
# Imports
import sympy as sym
import numpy as np
from IPython.display import display #for pretty printing

In [ ]:
# create symbolic variables

# system parameters
X0,Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of first link
g = sym.symbols('g')
m1,m2 = sym.symbols(['m1','m2']) # mass of links
l1,l2 = sym.symbols(['l1','l2']) # length of links
In1,In2 = sym.symbols(['In1','In2']) # moment of intertia of links

# generalized coordinates
th1,th2 = sym.symbols(['theta1','theta2']) #position
dth1,dth2 = sym.symbols([r'\dot{\\theta}_{1}',r'\dot{\\theta}_{2}']) #velocity
ddth1,ddth2 = sym.symbols([r'\ddot{\\theta}_{1}',r'\ddot{\\theta}_{2}']) #acceleration

q = sym.Matrix([[th1],[th2]]) #group into matrices
dq = sym.Matrix([[dth1],[dth2]])
ddq = sym.Matrix([[ddth1],[ddth2]])

display(ddq) #display prints it as cool latex stuff

## Positions

In [ ]:
# STEP 1: write expressions for the system space coordinates in terms of the generalized coordinates and parameters

th1a = th1
th2a = th2

##### Arm is at the origin so no need for X0 and Y0
xArm = X0+0.5*l1*sym.cos(th1a)
yArm = Y0+0.5*l1*sym.sin(th1a)
zArm = 0

xPen = l1*sym.cos(th1a) + 0.5*l2*(sym.sin(th2a)*sym.sin(th1a))
yPen = l1*sym.sin(th1a) - 0.5*l2*(sym.sin(th2a)*sym.cos(th1a))
zPen = l2*sym.cos(th2a)*0.5

## Velocities

### Linear velocities

In [ ]:
# STEP 2: generate expressions for the system space velocities
p1 = sym.Matrix([xArm,yArm,zArm,th1])
[dxArm,dyArm,dzArm,dth1a] = p1.jacobian(q)*dq

p2 = sym.Matrix([xPen,yPen,zPen,th2a])
[dxPen,dyPen,dzPen,dth2a] = p2.jacobian(q)*dq

dArm = sym.Matrix([dxArm, dyArm, dzArm])
dPen = sym.Matrix([dxPen, dyPen, dzPen])

display(dArm)
display(dPen)

### Angular velocities

In [ ]:
wArm = sym.Matrix([0, 0, dth1a])
wPen = sym.Matrix([dth2a, dth1a*sym.sin(th2a), dth1a*sym.cos(th2a)])

## Lagrange

In [ ]:
# STEP 3: generate expressions for the kinetic and potential energy
## Kinetic energies
Thub = 0
TArm = (0.5*m1*dArm.T*dArm) + (0.5*In1*wArm.T*wArm)
TPen = (0.5*m2*dPen.T*dPen) + (0.5*In2*wPen.T*wPen)
T = TArm + TPen
display(T)
#T = sym.Matrix([0.5*m1*(dxArm**2+dyArm**2) + 0.5*m2*(dxPen**2+dyPen**2) + 0.5*In1*dth1a**2 + 0.5*In2*dth2a**2])

## Potential energies
VArm = 0
VHub = 0
VPen = m2*g*zPen
V = sym.Matrix([VArm + VHub + VPen])
display(V)

In [ ]:
# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

#combine
EOM = Lg1 - Lg3 + Lg4

#display(EOM) #<-this will give you an idea of the magnitude of the faff a decent symbolic package lets you avoid

# Test

In [ ]:
#initial conditions
th10 = np.pi/4
th20 = 0
dth10 = 0
dth20 = 0

h = 0.01

#parameters
X0val = 0
Y0val = 2
l1val = 1
l2val = 1

parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(m1,1),(m2,1),(l1,l1val),(l2,l2val),(In1,0.08),(In2,0.08)]

th1ar = [th10]
th2ar = [th20]
dth1ar = [dth10]
dth2ar = [dth20]

N = 100
for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(th1,th1ar[i-1]),(th2,th2ar[i-1]),(dth1,dth1ar[i-1]),(dth2,dth2ar[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past)
    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddth1, ddth2])
    
    # integrate for the next velocity and position    
    dth1ar.append(float(dth1ar[i-1]+h*acc[ddth1]))
    dth2ar.append(float(dth2ar[i-1]+h*acc[ddth2]))
    th1ar.append(float(th1ar[i-1]+h*dth1ar[i]))
    th2ar.append(float(th2ar[i-1]+h*dth2ar[i]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Initialize the figure and 3D axis
fig1 = plt.figure()
ax1 = fig1.add_subplot(111, projection='3d')

# Define the lengths of the links
l1 = 1  # Length of the first link (arm)
l2 = 1  # Length of the second link (pendulum)

# Define the plot limits
ax1.set_xlim([-2, 2])
ax1.set_ylim([0, 4])
ax1.set_zlim([-2, 2])
ax1.set_xlabel('X axis')
ax1.set_ylabel('Y axis')
ax1.set_zlabel('Z axis')

def plot_furuta_pendulum_3d(i, th1_in, th2_in, ax):
    ax.clear()
    
    # Calculate positions of the arm
    L1topx = 0
    L1topy = 0
    L1topz = 0
    
    L1bottomx = l1 * np.cos(th1_in[i])
    L1bottomy = l1 * np.sin(th1_in[i])
    L1bottomz = 0
    
    # Plot the first link (arm)
    ax.plot([L1topx, L1bottomx], [L1topy, L1bottomy], [L1topz, L1bottomz], color='k', linewidth=2)
    
    # Add axis to the arm
    ax.quiver(L1topx, L1topy, L1topz, L1bottomx - L1topx, L1bottomy - L1topy, L1bottomz - L1topz, 
              color='r', length=l1, normalize=True, arrow_length_ratio=0.1)
    
    # Calculate positions of the pendulum
    # Pendulum rotates about the end of the arm
    L2bottomx = L1bottomx + l2 * np.cos(th1_in[i]) * np.cos(th2_in[i])
    L2bottomy = L1bottomy + l2 * np.sin(th1_in[i]) * np.cos(th2_in[i])
    L2bottomz = L1bottomz - l2 * np.sin(th2_in[i])
    
    # Plot the second link (pendulum)
    ax.plot([L1bottomx, L2bottomx], [L1bottomy, L2bottomy], [L1bottomz, L2bottomz], color='k', linewidth=2)
    
    # Add axis to the pendulum
    ax.quiver(L1bottomx, L1bottomy, L1bottomz, L2bottomx - L1bottomx, L2bottomy - L1bottomy, L2bottomz - L1bottomz, 
              color='b', length=l2, normalize=True, arrow_length_ratio=0.1)

    # Set plot limits
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([-2, 2])
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')

# Generate some example data
N = 100  # Number of frames
th1ar = np.linspace(0, 2 * np.pi, N)  # Example angle data for link 1
th2ar = np.linspace(0, np.pi, N)  # Example angle data for link 2

# Define the update function
update = lambda i: plot_furuta_pendulum_3d(i, th1ar, th2ar, ax1)

# Create the animation
animate = ani.FuncAnimation(fig1, update, frames=range(N), interval=50, repeat=True)

# Convert to HTML5 video for embedding in notebook
HTML(animate.to_html5_video())


In [ ]:
#initial conditions
th10 = np.pi/4
th20 = 0
dth10 = 0
dth20 = 0

h = 0.01

#parameters
X0val = 0
Y0val = 0
Z0val = 0
l1val = 1
l2val = 1

parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(m1,1),(m2,1),(l1,l1val),(l2,l2val),(In1,0.08),(In2,0.08)]

th1ar = [th10]
th2ar = [th20]
dth1ar = [dth10]
dth2ar = [dth20]

N = 100
for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(th1,th1ar[i-1]),(th2,th2ar[i-1]),(dth1,dth1ar[i-1]),(dth2,dth2ar[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past)
    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddth1, ddth2])
    
    # integrate for the next velocity and position    
    dth1ar.append(float(dth1ar[i-1]+h*acc[ddth1]))
    dth2ar.append(float(dth2ar[i-1]+h*acc[ddth2]))
    th1ar.append(float(th1ar[i-1]+h*dth1ar[i]))
    th2ar.append(float(th2ar[i-1]+h*dth2ar[i]))